# Preprocess

Preprocess the raw results from OntoGPT

Draft: https://docs.google.com/document/d/1H103ux6Dd1_bPM0un4RwutBLcYJx-0ybil2AwlAvG_Q/edit#

## Initial setup

Import libraries, create a GO adapter (for calculating closures/ancestors)

In [3]:
# note the gpt4 dir includes combined results from davinci, 3.5 and 4
results_dir = "results/human/gpt4"

In [4]:
import yaml
from yaml import Loader
from collections import defaultdict
import pandas as pd
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from ontogpt.evaluation.enrichment.eval_enrichment import EvalEnrichment
go = get_adapter("sqlite:obo:go")

/Users/marcin/Documents/VIMSS/ontology/LLMs/enrichgpt-results/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load the closure/ancestor map
# (takes a minute)
closure_map = defaultdict(set)
for s, _, o in go.relationships(predicates=[IS_A, PART_OF], include_entailed=True):
    closure_map[s].add(o)

In [6]:
print(len(closure_map))

84220


In [7]:
# ruamel is faster than pyyaml
from ruamel.yaml import YAML
ryaml = YAML()

## Load YAML results

See OntoGPT for details of data model

These results are nested, with a parent gene set object containing all runs.
We want to flatten this for analysis

In [9]:
from ontogpt.evaluation.enrichment.eval_enrichment import GeneSetComparison

In [10]:
# assumes comparisons have been run and concatenated (see Makefile) 
import glob
def load_gene_set_results():
    results = []
    for fn in glob.glob(f"{results_dir}/*.yaml"):
        print(fn)
        with open(fn) as f:
            #obj = yaml.load(f, Loader)
            #obj = yaml.safe_load(f)
            obj = ryaml.load(f)
            results.extend(obj)
    return results

In [11]:
comps = load_gene_set_results()

results/human/gpt4/canonical-glycolysis-gocam-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1001-results-2.yaml
results/human/gpt4/HALLMARK_HYPOXIA-results-2.yaml
results/human/gpt4/HALLMARK_DNA_REPAIR-results-2.yaml
results/human/gpt4/HALLMARK_G2M_CHECKPOINT-results-2.yaml
results/human/gpt4/EDS-results-2.yaml
results/human/gpt4/HALLMARK_IL2_STAT5_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION-results-2.yaml
results/human/gpt4/HALLMARK_PI3K_AKT_MTOR_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_COAGULATION-results-2.yaml
results/human/gpt4/peroxisome-results-2.yaml
results/human/gpt4/HALLMARK_APICAL_JUNCTION-results-2.yaml
results/human/gpt4/HALLMARK_ANGIOGENESIS-results-2.yaml
results/human/gpt4/go-postsynapse-calcium-results-2.yaml
results/human/gpt4/HALLMARK_BILE_ACID_METABOLISM-results-2.yaml
results/human/gpt4/HALLMARK_CHOLESTEROL_HOMEOSTASIS-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1002-results-2.yaml
results/huma

In [12]:
len(comps)

144

In [13]:
results_by_gene_sets = comps

## Create the main data frame

For each gene set, we will create a list of rows,
for each combination of method + parameters, plus a cutoff.

A cutoff of zero means we only consider the *top rank* GO term from enrichment.
If the results contain this, it's a true positive; otherwise we count as a single false positive.

Otherwise we look at all enrichment results with p-val less than the cutoff. If predicted terms match these,
it's a true positive.

For calculating false negatives, we check both ancestors and descendants

In [17]:
import itertools
pairs = list(itertools.product([0.005, 0.05, 99], [False, True], [1, 5, 10, 25, 5000]))

def eval_gene_set_result(gs):
    """
    For each gene set makes, sets of rows, for each combination of method/params,
    plus each combination of p-value cutoff
    """
    rows = []
    std = gs["payloads"]["standard"]
    if "enrichment_results" not in std:
        print(f"NO GOLD STANDARD: {gs['name']}")
    expected_results = std.get("enrichment_results", [])
    for run, method_result in gs["payloads"].items():
        predicted_term_ids = method_result.get("term_ids", [])
        #closure_map = defaultdict(list)
        predicted_term_closure_ids = set()
        for t in predicted_term_ids:
            predicted_term_closure_ids.update(closure_map.get(t, set()))
        #for s, _, o in go.relationships(subjects=predicted_term_ids, predicates=[IS_A, PART_OF], include_entailed=True):
            #closure_map[s].append(o)
        #    predicted_term_closure_ids.add(o)
        for cutoff, use_closure, top_n in pairs:
            if top_n == 1 and cutoff != 0.05:
                continue
            row = {}
            row["name"] = f"{gs['name']}-{cutoff}"
            row["gene_set"] = gs["name"]
            row["cutoff"] = cutoff
            row["closure"] = use_closure
            row["top_n"] = top_n
            method = method_result.get("method", "")
            approach = "gpt"
            if method == "no_synopsis":
                src = "NONE"
            elif method == "ontological_synopsis":
                src = "GO"
            elif method == "narrative_synopsis":
                src = "RefSeq"
            else:
                approach = method
                src = ""
            row["source"] = src
            model = method_result.get("model", "")
            if model == "gpt-4":
                model = "4"
            elif model == "gpt-3.5-turbo":
                model = "3.5"
            elif model == "text-davinci-003":
                model = "3"
            row["model"] = model
            row["method"] = approach
            if model:
                row["method_desc"] = f"{method}-{model}"
            else:
                row["method_desc"] = method_result.get("truncation_factor", "")
            row["run"] = run
            for k in ["truncation_factor", "prompt_variant", "response_token_length"]:
                row[k] = method_result.get(k, "")
            row["prompt_length"] = len(method_result.get("prompt", ""))
            true_positive_terms = []
            false_negative_terms = []
            more_specific_false_negative_terms = []   # predicted a descendant
            more_general_false_negative_terms = []   # predicted a descendant
            unparsed_terms = []
            standard_enrichment_results = [(r["p_value_adjusted"], r["class_id"]) for r in expected_results]
            enrichment_closure = set()
            enrichment_term_ids = set()
            if cutoff == 99:
                # extend the enrichment results but with max cutoff
                for x in gs["payloads"]["closure"]["term_ids"]:
                    standard_enrichment_results.append((cutoff, x))
            visited_terms = set()
            n = 0
            for p_val, true_term_id in standard_enrichment_results:
                if cutoff > 0 and p_val > cutoff:
                    break
                if true_term_id in visited_terms:
                    continue
                n += 1
                if n > top_n:
                    break
                enrichment_closure.update(closure_map.get(true_term_id, set()))
                enrichment_term_ids.add(true_term_id)
                true_term_closure_ids = closure_map.get(true_term_id, set())
                if use_closure:
                    visited_terms.update(true_term_closure_ids)
                else:
                    visited_terms.add(true_term_id)
                if true_term_id in predicted_term_ids:
                    true_positive_terms.append(true_term_id)
                elif true_term_id in predicted_term_closure_ids:
                    # predicted a more specific term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_specific_false_negative_terms.append(true_term_id)
                elif true_term_closure_ids.intersection(predicted_term_ids):
                    # predicted a more general term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_general_false_negative_terms.append(true_term_id)
                else:
                    false_negative_terms.append(true_term_id)
                if cutoff == 0:
                    break
            false_positive_terms = []
            if top_n > 1:
                for t in predicted_term_ids:
                    if t not in true_positive_terms:
                        if t.startswith("GO:"):
                            if not use_closure:
                                false_positive_terms.append(t)
                            else:
                                if t in enrichment_closure:
                                    # prediction is more general
                                    pass
                                elif closure_map.get(t, set()).intersection(enrichment_term_ids):
                                    # prediction is more specific
                                    pass
                                else:
                                    false_positive_terms.append(t)
                        elif ":" in t:
                            # MONDO, UBERON, etc
                            pass
                        else:
                            unparsed_terms.append(t)
            else:
                if not true_positive_terms:
                    if predicted_term_ids:
                        false_positive_terms.append(predicted_term_ids[0])
            row["true_positives"] = len(true_positive_terms)
            row["false_positives"] = len(false_positive_terms)
            row["false_negatives"] = len(false_negative_terms)
            row["more_general_false_negatives"] = len(more_general_false_negative_terms)
            row["more_specific_false_negatives"] = len(more_specific_false_negative_terms)
            row["all_predictions_closure"] = len(predicted_term_closure_ids)
            row["unparsed"] = len(unparsed_terms)
            row["true_positive_terms"] = "|".join(true_positive_terms)
            row["false_positive_terms"] = "|".join(false_positive_terms)
            row["unparsed_terms"] = "|".join(unparsed_terms)
            row["gene_set_size"] = len(gs.get("gene_symbols"))
            
            denom1 = row["true_positives"] + row["false_positives"]
            if denom1 != 0:
                row["precision"] = row["true_positives"] / denom1
            else:
                row["precision"] = 0

            denom2 = row["true_positives"] + row["false_positives"]
            if denom2 != 0:
                row["recall"] = row["true_positives"] / denom2
            else:
                row["recall"] = 0
                
            denom3 = row["true_positives"] + row["more_general_false_negatives"]
            if denom3 != 0:
                row["recall_general"] = row["true_positives"] / denom3
            else:
                row["recall_general"] = 0
                
            denom4 = row["true_positives"] + row["more_specific_false_negatives"]
            if denom4 != 0:
                row["recall_specific"] = row["true_positives"] / denom4
            else:
                row["recall_specific"] = 0
                
            #row["gene_symbols"] = "|".join(gs.get("gene_symbols", []))
            #row["gene_ids"] = "|".join(gs.get("gene_ids", []))
            rows.append(row)
    return rows

rows = eval_gene_set_result(results_by_gene_sets[4])
df = pd.DataFrame(rows)
df.to_csv("results/TEST-processed.tsv", sep="\t", index=False)
df

,name,gene_set,cutoff,closure,top_n,source,model,method,method_desc,run,...,all_predictions_closure,unparsed,true_positive_terms,false_positive_terms,unparsed_terms,gene_set_size,precision,recall,recall_general,recall_specific
0,HALLMARK_HYPOXIA-0-0.005,HALLMARK_HYPOXIA-0,0.005,False,5,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,53,4,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,oxidative stress response|apoptosis regulation...,200,0.333333,0.333333,0.500000,0.666667
1,HALLMARK_HYPOXIA-0-0.005,HALLMARK_HYPOXIA-0,0.005,False,10,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,53,4,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,oxidative stress response|apoptosis regulation...,200,0.333333,0.333333,0.333333,0.500000
2,HALLMARK_HYPOXIA-0-0.005,HALLMARK_HYPOXIA-0,0.005,False,25,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,53,4,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,oxidative stress response|apoptosis regulation...,200,0.333333,0.333333,0.133333,0.400000
3,HALLMARK_HYPOXIA-0-0.005,HALLMARK_HYPOXIA-0,0.005,False,5000,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,53,4,GO:0005975|GO:0006096|GO:0001525,GO:1904659|GO:0051726|GO:0030198,oxidative stress response|apoptosis regulation...,200,0.500000,0.500000,0.125000,0.096774
4,HALLMARK_HYPOXIA-0-0.005,HALLMARK_HYPOXIA-0,0.005,True,5,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,53,4,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:1904659|GO:0051726|GO:0001525|GO:0030198,oxidative stress response|apoptosis regulation...,200,0.555556,0.555556,0.714286,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,HALLMARK_HYPOXIA-0-99,HALLMARK_HYPOXIA-0,99.000,False,5000,,,,,closure,...,5699,0,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0006403|GO:0015931|GO:0050657|GO:0050658|GO...,,200,0.879507,0.879507,1.000000,1.000000
594,HALLMARK_HYPOXIA-0-99,HALLMARK_HYPOXIA-0,99.000,True,5,,,,,closure,...,5699,0,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0006011|GO...,,200,0.000616,0.000616,1.000000,1.000000
595,HALLMARK_HYPOXIA-0-99,HALLMARK_HYPOXIA-0,99.000,True,10,,,,,closure,...,5699,0,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0006011|GO...,,200,0.001235,0.001235,1.000000,1.000000
596,HALLMARK_HYPOXIA-0-99,HALLMARK_HYPOXIA-0,99.000,True,25,,,,,closure,...,5699,0,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0061975|GO...,,200,0.003627,0.003627,1.000000,1.000000


In [16]:
rows = []
for gs in results_by_gene_sets:
    this_rows = eval_gene_set_result(gs)
    rows.extend(this_rows)
    print(len(rows))
    df = pd.DataFrame(rows)
    df.to_csv("results/TEMP.tsv", sep="\t", index=False)
df = pd.DataFrame(rows)
pd.set_option('display.max_rows', 10)
df.to_csv("results/processed.tsv", sep="\t", index=False)
df

598
1196
NO GOLD STANDARD: bicluster_RNAseqDB_1001-0
1794
NO GOLD STANDARD: bicluster_RNAseqDB_1001-1
2392
2990
3588
4186
4784
5382
5980
6578
7176
7774
8372
8970
9568
10166
10764
11362
11960
12558
13156
13754
14352
14950
15548
16146
16744
17342
17940
18538
19136
19734
20332
20930


KeyboardInterrupt: 

In [ ]:
df["method"].unique()